## Графовые нейронные сети (GNNs)

Графовые нейронные сети представляют собой класс нейронных сетей, специально разработанный для работы с графовыми структурами данных. Они могут моделировать зависимости и взаимодействия между сущностями в графе, позволяя учитывать контекст и структуру данных при выполнении задач.

В GNN каждая вершина графа обновляется на основе её соседей, что позволяет модели учить представления вершин, учитывая их окружение. Такие модели широко применяются в задачах, связанных с анализом социальных сетей, биологических сетей, рекомендательных систем и графовых знаний.

## Graph embedding
Эмбеддинги в контексте графовых нейронных сетей представляют собой векторные представления вершин графа. Они извлекаются из GNN и представляют сущности в пространстве низкой размерности. Эти векторы обладают свойством сохранять структурную информацию о графе и могут использоваться для различных задач, таких как предсказание отношений или классификация вершин.

## Тестирование и Валидация

Тестирование и валидация важны для оценки работы графовых нейронных сетей.

Тестирование позволяет оценить обобщающую способность модели на новых данных, которые не участвовали в обучении. Это важно для проверки, насколько хорошо модель обобщает свои знания на новые сценарии.

Валидация используется для подбора оптимальных гиперпараметров модели. Настройка параметров происходит на основе производительности модели на отложенной валидационной выборке.

### Установка зависимостей

Для работы с графовыми нейронными сетями будем использовать [Pykeen](https://pykeen.readthedocs.io/en/stable/). Необходимо импортировать соответствующую библиотеку. В данном шаге мы устанавливаем Pykeen. Этот шаг включает установку библиотеки Pykeen с использованием команды !pip install pykeen. Pykeen предоставляет реализацию различных моделей графовых знаний и инструменты для работы с ними.

In [1]:
# Установка Pykeen
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.3/739.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.7 MB/s eta 0:00:00
  Created wheel for rexmex: filename=rexmex-0.1.3-py3-none-any.whl size=19952 sha256=773f5fad3e4b6726cbdaf14c2d5ab14bc84b5a6ccb644e9977ad568fa03a5f36
  Stored in directory: /root/.cache/pip/wheels/13/31/9d/51fb64454e6012bc88dd72b57609646432e478ba35772736d6
Successfully built rexmex


Далее, мы импортируем библиотеку Pykeen для использования её функциональности в нашем коде.

In [2]:
import pykeen

После выполнения этого шага, мы готовы к использованию Pykeen для создания, обучения и оценки моделей графовых знаний.

## Взаимодействие с графовым датасетом и описание данных

В данной работе мы будем использовать графовый датасет [Nations](https://paperswithcode.com/dataset/nations), предоставляющий информацию о связях между стран и их политических отношений. Этот небольшой датасет хорошо подходит для погружения в задачу предсказания отношений между сущностями.


Импорт необходимого модуля Pykeen. Мы импортируем модуль datasets из Pykeen для работы с графовыми датасетами.

In [3]:
from pykeen.datasets import PharmKG

Загружаем графовый датасет Nations из Pykeen. В результате выполнения этой команды будет создан объект, представляющий графовый датасет.

In [5]:
# Загрузка графового датасета Nations
pharm_dataset = PharmKG()

Мы можем вывести описание датасета, чтобы получить информацию о количестве сущностей, отношений и других характеристиках.

In [6]:
# Просмотр описания датасета
print(pharm_dataset)

INFO:pykeen.datasets.base:downloading data from https://zenodo.org/record/4077338/files/raw_PharmKG-180k.zip to /root/.data/pykeen/datasets/pharmkg/raw_PharmKG-180k.zip


INFO:pykeen.datasets.base:reordering columns: ['Entity1_name', 'relationship_type', 'Entity2_name']
INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [691412, 109324, 109324]


PharmKG(num_entities=188296, num_relations=39, create_inverse_triples=False)


#### Получение тренировочных данных
Мы можем получить тренировочные факты, представленные в виде троек сущность-отношение-сущность.
Таким образом, после выполнения этих шагов, у нас есть загруженный графовый датасет, и мы можем использовать его для обучения и тестирования моделей графовых знаний в Pykeen.

In [7]:
# Получение тренировочных фактов (троек сущность-отношение-сущность)
training_data = pharm_dataset.training.mapped_triples
print(training_data[:5])

tensor([[     0,      2, 161966],
        [     0,     21,  26679],
        [     1,     17,  95159],
        [     2,     31,  23633],
        [     3,      3,  65653]])


## Описание базовой задачи

В данном контексте базовой задачей является предсказание отношений между сущностями в графовом датасете. Каждая тройка в графе представляет отношение между двумя сущностями. Например, в социальной сети, где сущности - пользователи и страницы, отношения могут быть "пользователь подписан на страницу".

In [8]:
# Импорт необходимых модулей Pykeen
from pykeen.pipeline import pipeline


#### Выбор модели
Мы выбираем модель для решения задачи предсказания отношений. В данном случае, мы используем модель [TransE](https://paperswithcode.com/method/transe), одну из популярных моделей для этой задачи.

#### Выбор датасета
Мы указываем графовый датасет, с которым будем работать. В данном примере - nations_dataset, который содержит данные о связях в между странами.

#### Количество эпох обучения
Задаем количество эпох обучения (итераций по всему датасету).

Проверяем наличие GPU устройства и вызываем функцию pipeline(), которая запускает процесс обучения модели, тестирования и валидации.

In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [10]:
# Задаем параметры для pipeline
pipeline_results = pipeline(
    model="TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset=nations_dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs=dict(num_epochs=20),   # Количество эпох обучения
    device=device
)

INFO:pykeen.datasets.base:reordering columns: ['Entity1_name', 'relationship_type', 'Entity2_name']
INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [691412, 109324, 109324]
INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=128.


Evaluating on cuda:0:   0%|          | 0.00/109k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 216.19s seconds



Этот код инициирует обучение модели на выбранном датасете, используя выбранную архитектуру (TransE) и указанные параметры. После завершения обучения, результаты выводятся, и можно провести анализ производительности модели.


Печатаем результаты, которые могут включать в себя метрики производительности, потери на тренировочных данных, а также результаты на тестовом и валидационном наборах данных.

In [11]:
# Выводим результаты
print(pipeline_results)

PipelineResult(random_seed=1333030492, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(188296, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(39, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=188296, num_relations=39, create_inverse_triples=False, num_triples=874588, path="/root/.data/pykeen/datasets/pharmkg/raw_PharmKG-180k.zip"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7974bd973e20>, losses=[0.6948997225979807, 0.26703097181760466, 0.16985537947959847, 0.13118342844290207, 0.10990948250524335, 0.09551327764815691, 0.08566794139658773, 0.07774332151109097, 0.07134034464891835, 0.0656657024164855, 0.06209576333621405, 0.0586199894883251, 0.05568090238206946, 0.0530647124957761, 0.05109510084634327, 

### Triple Scoring

Импортируем [predict_triples](https://pykeen.readthedocs.io/en/stable/api/pykeen.predict.predict_triples.html)

In [12]:
from pykeen.predict import predict_triples

Рассчитаем оценки для всех validation троек из набора данных, на котором мы обучали модель.

In [13]:
pack = predict_triples(model=pipeline_results.model, triples=pharm_dataset.validation)
df = pack.process(factory=pipeline_results.training).df



```
# This is formatted as code
```

Показаны тройки, получившие наивысший score

In [14]:
df.nlargest(n=5, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
3151,33806,bdnf nt 3 growth factors receptor,16,I,33806,bdnf nt 3 growth factors receptor,-0.6759
9185,72841,fbxl7,16,I,72841,fbxl7,-0.6759
14283,88528,hspa4l,16,I,88528,hspa4l,-0.6759
16691,21656,aldh4a1,16,I,21656,aldh4a1,-0.6759
18749,154021,rhoq,16,I,154021,rhoq,-0.6759


В рамках самостоятельной работы попробуйте обучить и протестировать модель на любом другом датасете доступном на https://pykeen.readthedocs.io/en/stable/reference/datasets.html.